In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-125056
Azure region: southcentralus
Subscription id: 82648f26-b738-43a4-9ebb-f954c9f1ff3a
Resource group: aml-quickstarts-125056


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import normal, loguniform, choice
import shutil
import os

# Specify parameter sampler

ps = RandomParameterSampling( {
        "--C" : choice(1,2,4),
        "--max_iter" : choice(10,50,100)
      
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2)

if "training" not in os.listdir():
    os.mkdir("./training")

##shutil.copy('train.py', "./training")
    
    
# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = '.', 
              entry_script = 'train.py', 
              compute_target = cpu_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=12,
                                     max_concurrent_runs=4,
                                     policy = policy,
                                     estimator = est)

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_c82fa062-996d-4ab1-813c-e4e679c1383b
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_c82fa062-996d-4ab1-813c-e4e679c1383b?wsid=/subscriptions/82648f26-b738-43a4-9ebb-f954c9f1ff3a/resourcegroups/aml-quickstarts-125056/workspaces/quick-starts-ws-125056

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-07T15:10:30.976414][API][INFO]Experiment created<END>\n""<START>[2020-11-07T15:10:31.485996][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-11-07T15:10:31.736659][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-11-07T15:10:32.3949891Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_c82fa062-996d-4ab1-813c-e4e679c1383b
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_c82fa062-996d-4ab1-813c-e4e679c1383b?wsid=/subscriptions/82648

In [6]:
import joblib

# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()


In [7]:
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())


['--C', '2', '--max_iter', '50']
['azureml-logs/55_azureml-execution-tvmps_eeb8df38a236b5c2fb91aef973fad2c4c24fff947afce7dc7b025679aeba8f91_d.txt', 'azureml-logs/65_job_prep-tvmps_eeb8df38a236b5c2fb91aef973fad2c4c24fff947afce7dc7b025679aeba8f91_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_eeb8df38a236b5c2fb91aef973fad2c4c24fff947afce7dc7b025679aeba8f91_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/109_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']


In [8]:

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
best_run


Best Run Id:  HD_c82fa062-996d-4ab1-813c-e4e679c1383b_0

 Accuracy: 0.9109256449165403


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_c82fa062-996d-4ab1-813c-e4e679c1383b_0,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [9]:
best_run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_eeb8df38a236b5c2fb91aef973fad2c4c24fff947afce7dc7b025679aeba8f91_d.txt',
 'azureml-logs/65_job_prep-tvmps_eeb8df38a236b5c2fb91aef973fad2c4c24fff947afce7dc7b025679aeba8f91_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_eeb8df38a236b5c2fb91aef973fad2c4c24fff947afce7dc7b025679aeba8f91_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/109_azureml.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/model.joblib']

In [10]:
model = best_run.register_model(model_name='model_project', model_path='outputs/model.joblib')

In [12]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = Dataset.Tabular.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [13]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [24]:
import pandas as pd

y_dataframe = pd.DataFrame(y, columns=['y'])
y_dataframe.head()

,y
0,0
1,0
2,0
3,0
4,0


In [27]:
all_data = pd.concat([x, y_dataframe], axis=1)
all_data.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0


In [31]:
from sklearn.model_selection import train_test_split

x_train, x_test= train_test_split(all_data, train_size=0.80,test_size=0.20, random_state=1, shuffle=True)



In [32]:
x_train.to_csv("training/train_data.csv")


In [33]:
ds = ws.get_default_datastore()
ds

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-b2fd962f-a2a5-4722-8f4d-24f34ed95ffa",
  "account_name": "mlstrg125056",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [34]:
ds.upload(src_dir='training/', target_path='bankmarketing', overwrite=True, show_progress=True)

Uploading an estimated of 1 files
Uploading training/train_data.csv
Uploaded training/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_bf04d5b56b8141d7b21be7e3dab63fbc

In [35]:
train_data = Dataset.Tabular.from_delimited_files(path=ds.path('bankmarketing/train_data.csv'))

In [36]:
from azureml.train.automl import AutoMLConfig
label = "y"
# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=train_data,
    label_column_name=label,
    n_cross_validations=5,
    compute_target = cpu_cluster)

In [37]:
# Submit your automl run
from azureml.widgets import RunDetails
remote_run = exp.submit(automl_config, show_output = True)
RunDetails(automl_run).show()
remote_run.wait_for_completion(show_output=True)

Running on remote.
Running on remote compute: cpucluster
Parent Run ID: AutoML_9398f8f8-b2fa-4538-89c4-82782f1257bb

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class      

In [41]:
# Retrieve and save your best automl model.
best_run_customized, best_model = remote_run.get_output()

In [42]:
best_run_customized.register_model(model_name = "automlproject.pkl", model_path = './outputs/')
print(best_model._final_estimator)

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('0',
                                           Pipeline(memory=None,
                                                    steps=[('maxabsscaler',
                                                            MaxAbsScaler(copy=True)),
                                                           ('lightgbmclassifier',
                                                            LightGBMClassifier(boosting_type='gbdt',
                                                                               class_weight=None,
                                                                               colsample_bytree=1.0,
                                                                               importance_type='split',
                                                                               learning_rate=0.1,
                                                                               max_

In [ ]:
cp_cluster.delete()